<a href="https://colab.research.google.com/github/maorgreshler/NN_google_colab/blob/main/overfitcheck.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import io
from google.colab import files
uploaded = files.upload()

df = pd.read_csv(io.BytesIO(uploaded['vehicle_simulation_data_random_test.csv']))
print(df.head())

In [58]:
input_columns = ["velocity", "yaw_angle", "yaw_rate", "slip_angle", "front_wheel", "back_wheel" , "acceleration", "angular_acceleration"]

output_columns = ["delta_x_position", "delta_y_position", "delta_theta", "delta_velocity", "delta_yaw_angle",
                  "delta_yaw_rate", "delta_slip_angle", "delta_front_wheel", "delta_back_wheel"]

input_data = df[input_columns]
output_data = df[output_columns]

In [64]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Convert pandas DataFrames to numpy arrays
X = input_data.values
y = output_data.values
# Normalize the data
scaler_X = StandardScaler()
scaler_y = StandardScaler()

X = scaler_X.fit_transform(X)
y = scaler_y.fit_transform(y)

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Convert numpy arrays to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_size, output_size):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(input_size, 64)
        self.fc2 = nn.Linear(64, 128)
        self.fc3 = nn.Linear(128, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# Instantiate the model, define loss function and optimizer
input_size = X_train.shape[1]
output_size = y_train.shape[1]
model = MLP(input_size, output_size)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Train the model
epochs = 200
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    loss = criterion(outputs, y_train_tensor)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{epochs}], Loss: {loss.item():.4f}')

Epoch [10/200], Loss: 0.8628
Epoch [20/200], Loss: 0.6427
Epoch [30/200], Loss: 0.4090
Epoch [40/200], Loss: 0.2172
Epoch [50/200], Loss: 0.1068
Epoch [60/200], Loss: 0.0477
Epoch [70/200], Loss: 0.0148
Epoch [80/200], Loss: 0.0034
Epoch [90/200], Loss: 0.0006
Epoch [100/200], Loss: 0.0004
Epoch [110/200], Loss: 0.0002
Epoch [120/200], Loss: 0.0000
Epoch [130/200], Loss: 0.0000
Epoch [140/200], Loss: 0.0000
Epoch [150/200], Loss: 0.0000
Epoch [160/200], Loss: 0.0000
Epoch [170/200], Loss: 0.0000
Epoch [180/200], Loss: 0.0000
Epoch [190/200], Loss: 0.0000
Epoch [200/200], Loss: 0.0000


In [65]:
# Evaluate the model
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor)
    test_loss = criterion(test_outputs, y_test_tensor)
    print(f'Test Loss: {test_loss.item():.4f}')

#my overfit!!!
# Evaluate the model
model.eval()
with torch.no_grad():
    train_outputs = model(X_train_tensor)
    train_loss = criterion(train_outputs, y_train_tensor)
    print(f'Train Loss: {train_loss.item():.4f}')


Test Loss: 0.9875
Train Loss: 0.0000


In [66]:
# Make predictions
with torch.no_grad():
    predictions = model(X_test_tensor)
    predictions = scaler_y.inverse_transform(predictions.numpy())

print("Predictions:")
print(predictions)

Predictions:
[[-0.032333 -0.011169 -0.000272  0.04358  -0.001771  0.032075  0.007467
   1.008607 -6.188357]]


In [71]:
#my overfit!!!
# Make predictions
with torch.no_grad():
    predictions = model(X_train_tensor)
    predictions = scaler_y.inverse_transform(predictions.numpy())

print("Predictions:")
import numpy as np
np.set_printoptions(suppress=True, precision=6)
print(predictions[0])

Predictions:
[ 0.014616  0.02139  -0.        0.084641 -0.001935 -0.006616 -0.007681
 -0.636284 -5.106743]
